In [1]:
import os
import numpy as np
import pandas as pd
import warnings

# This is not recommended but I am doing this to suppress warnings from SARIMAX
warnings.simplefilter('ignore')

countryName = 'Germany'
nFeatures = 1

nDaysMin = 15

nValid = 10
nTest = 10

dataDir = os.path.join('data', 'JHU', 'upto05252020_forPublication')


confirmedFilename = 'time_series_covid19_confirmed_global.csv'
deathsFilename = 'time_series_covid19_deaths_global.csv'
recoveredFilename = 'time_series_covid19_recovered_global.csv'

In [2]:
# split a univariate sequence into samples
def split_sequence(sequence, n_steps):
    X, y = list(), list()
    for i in range(len(sequence)):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the sequence
        if end_ix > len(sequence)-1:
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

In [3]:
def meanAbsolutePercentageError(yTrueList, yPredList):
    absErrorList = [np.abs(yTrue - yPred) for yTrue, yPred in zip(yTrueList, yPredList)]
    absPcErrorList = [absError/yTrue for absError, yTrue in zip(absErrorList, yTrueList)]
    MAPE = 100*np.mean(absPcErrorList)
    return MAPE

def meanForecastError(yTrueList, yPredList):
    forecastErrors = [yTrue - yPred for yTrue, yPred in zip(yTrueList, yPredList)]
    MFE = np.mean(forecastErrors)
    return MFE

def meanAbsoluteError(yTrueList, yPredList):
    absErrorList = [np.abs(yTrue - yPred) for yTrue, yPred in zip(yTrueList, yPredList)]
    return np.mean(absErrorList)

def meanSquaredError(yTrueList, yPredList):
    sqErrorList = [np.square(yTrue - yPred) for yTrue, yPred in zip(yTrueList, yPredList)]
    return np.mean(sqErrorList)

def rootMeanSquaredError(yTrueList, yPredList):
    return np.sqrt(meanSquaredError(yTrueList, yPredList))

def medianSymmetricAccuracy(yTrueList, yPredList):
    '''https://helda.helsinki.fi//bitstream/handle/10138/312261/2017SW001669.pdf?sequence=1'''
    logAccRatioList = [np.abs(np.log(yPred/yTrue)) for yTrue, yPred in zip(yTrueList, yPredList)]
    MdSA = 100*(np.exp(np.median(logAccRatioList))-1)
    return MdSA

In [4]:
# Function to get all three frames for a given country
def getCountryCovidFrDict(countryName):
    countryCovidFrDict = {}
    for key in covidFrDict.keys():
        dataFr = covidFrDict[key]
        countryCovidFrDict[key] = dataFr[dataFr['Country/Region'] == countryName]
    return countryCovidFrDict

In [5]:
# Load all 3 csv files
covidFrDict = {}
covidFrDict['confirmed'] = pd.read_csv(os.path.join(dataDir, confirmedFilename))
covidFrDict['deaths'] = pd.read_csv(os.path.join(dataDir, deathsFilename))

# Recovered is back again!
covidFrDict['recovered'] = pd.read_csv(os.path.join(dataDir, recoveredFilename))

countryCovidFrDict = getCountryCovidFrDict(countryName)

# Get list of dates
colNamesList = list(countryCovidFrDict['confirmed'])
dateList = [colName for colName in colNamesList if '/20' in colName]
dataList = [countryCovidFrDict['confirmed'][date].iloc[0] for date in dateList]
dataDict = dict(zip(dateList, dataList))

# Get time series for cases > 100 only
daysSince = 100
nCasesGreaterDaysSinceList = []
datesGreaterDaysSinceList = []

for key in dataDict.keys():
    if dataDict[key] > daysSince:
        datesGreaterDaysSinceList.append(key)
        nCasesGreaterDaysSinceList.append(dataDict[key])
        
XList, yList = split_sequence(nCasesGreaterDaysSinceList, nDaysMin)

XTrainList = XList[0:len(XList)-(nValid + nTest)]
XValidList = XList[len(XList)-(nValid+nTest):len(XList)-(nTest)]
XTestList = XList[-nTest:]

yTrain = yList[0:len(XList)-(nValid + nTest)]
yValid = yList[len(XList)-(nValid+nTest):len(XList)-(nTest)]
yTest = yList[-nTest:]

print('Total size of data points for LSTM:', len(yList))
print('Size of training set:', len(yTrain))
print('Size of validation set:', len(yValid))
print('Size of test set:', len(yTest))

# Convert from list to matrix
XTrain = XTrainList.reshape((XTrainList.shape[0], XTrainList.shape[1], nFeatures))
XValid = XValidList.reshape((XValidList.shape[0], XValidList.shape[1], nFeatures))
XTest = XTestList.reshape((XTestList.shape[0], XTestList.shape[1], nFeatures))

print(XTrain.shape)

Total size of data points for LSTM: 71
Size of training set: 51
Size of validation set: 10
Size of test set: 10
(51, 15, 1)


# Vanilla LSTM

In [6]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import Adam

nNeurons = 100
nFeatures = 1

bestValidMAPE = 100
bestSeed = -1
for seed in range(100):
    tf.random.set_seed(seed=seed)
    
    # define model
    model = Sequential()
    model.add(LSTM(nNeurons, activation='relu', input_shape=(nDaysMin, nFeatures)))
    model.add(Dense(1))
    opt = Adam(learning_rate=0.1)
    model.compile(optimizer=opt, loss='mse')

    # fit model
    model.fit(XTrain, yTrain, epochs=1000, verbose=0)

    yPred = list(model.predict(XValid, verbose=0))
    yPredList = []
    for i in range(len(yPred)):
        yPredList.append(yPred[i][0])

#     for yTrue, yPred in zip(yTest, yPredList):
#         print(yTrue, yPred)

    MAPE = meanAbsolutePercentageError(yValid, yPredList)
    print(seed, MAPE)
    if MAPE < bestValidMAPE:
        print('Updating best MAPE to {}...'.format(MAPE))
        bestValidMAPE = MAPE
        print('Updating best seed to {}...'.format(seed))
        bestSeed = seed

# define model
print('Training model with best seed...')
tf.random.set_seed(seed=bestSeed)
model = Sequential()
model.add(LSTM(nNeurons, activation='relu', input_shape=(nDaysMin, nFeatures)))
model.add(Dense(1))
opt = Adam(learning_rate=0.1)
model.compile(optimizer=opt, loss='mse')

# fit model
model.fit(XTrain, yTrain, epochs=1000, verbose=0)

yPred = list(model.predict(XTest, verbose=0))
yPredList = []
for i in range(len(yPred)):
    yPredList.append(yPred[i][0])
    
MAPE = meanAbsolutePercentageError(yTest, yPredList)
print('Test MAPE:', MAPE)
MdSA = medianSymmetricAccuracy(yTest, yPredList)
print('Test MdSA:', MdSA)

AttributeError: module 'tensorflow._api.v1.random' has no attribute 'set_seed'

# Stacked LSTM

In [7]:
# define model
nNeurons = 50
nFeatures = 1

bestValidMAPE = 100
bestSeed = -1
for seed in range(100):
    tf.random.set_seed(seed=seed)
    model = Sequential()
    model.add(LSTM(nNeurons, activation='relu', return_sequences=True, input_shape=(nDaysMin, nFeatures)))
    model.add(LSTM(nNeurons, activation='relu'))
    model.add(Dense(1))
    opt = Adam(learning_rate=0.1)
    model.compile(optimizer=opt, loss='mse')

    # fit model
    model.fit(XTrain, yTrain, epochs=1000, verbose=0)

    yPred = list(model.predict(XValid, verbose=0))
    yPredList = []
    for i in range(len(yPred)):
        yPredList.append(yPred[i][0])

#     for yTrue, yPred in zip(yTest, yPredList):
#         print(yTrue, yPred)

    MAPE = meanAbsolutePercentageError(yValid, yPredList)
    print(seed, MAPE)
    if MAPE < bestValidMAPE:
        print('Updating best MAPE to {}...'.format(MAPE))
        bestValidMAPE = MAPE
        print('Updating best seed to {}...'.format(seed))
        bestSeed = seed

# define model
print('Training model with best seed...')
tf.random.set_seed(seed=bestSeed)
model = Sequential()
model.add(LSTM(nNeurons, activation='relu', return_sequences=True, input_shape=(nDaysMin, nFeatures)))
model.add(LSTM(nNeurons, activation='relu'))
model.add(Dense(1))
opt = Adam(learning_rate=0.1)
model.compile(optimizer=opt, loss='mse')

# fit model
model.fit(XTrain, yTrain, epochs=1000, verbose=0)

yPred = list(model.predict(XTest, verbose=0))
yPredList = []
for i in range(len(yPred)):
    yPredList.append(yPred[i][0])
    
MAPE = meanAbsolutePercentageError(yTest, yPredList)
print('Test MAPE:', MAPE)
MdSA = medianSymmetricAccuracy(yTest, yPredList)
print('Test MdSA:', MdSA)

0 99.96471695355943
Updating best MAPE to 99.96471695355943...
Updating best seed to 0...
1 99.96707537204928
2 2.1818614606414664
Updating best MAPE to 2.1818614606414664...
Updating best seed to 2...
3 0.13937600025860616
Updating best MAPE to 0.13937600025860616...
Updating best seed to 3...
4 3.9151496142667352
5 99.97184959349138
6 99.98955903407396
7 99.98920229111722
8 99.99171161651014
9 99.99961895742317
10 99.89099478618921
11 13.141272136808254
12 99.9666796932781
13 99.95745330084716
14 1.3920001465120218
15 2.8888679998589333
16 1.839477382514299
17 0.9618431926379203
18 99.97504327914586
19 2.9597795208125706
20 99.9661518067712
21 99.96269818395673
22 99.95656729574381
23 11.220099032664107
24 49.62313864798829
25 99.95797106903052
26 39.61193930040988
27 1.1651038538571956
28 nan
29 nan
30 99.99983759030033
31 99.96463103427182
32 17.111583496232868
33 99.99684564145798
34 99.89632062475505
35 100.00007368708536
36 99.9600790811827
37 99.98785278742773
38 99.99908957064

# Bidirectional LSTM

In [8]:
from tensorflow.keras.layers import Bidirectional

# define model
nNeurons = 50
nFeatures = 1

bestValidMAPE = 100
bestSeed = -1
for seed in range(100):
    tf.random.set_seed(seed=seed)
    model = Sequential()
    model.add(Bidirectional(LSTM(nNeurons, activation='relu'), input_shape=(nDaysMin, nFeatures)))
    model.add(Dense(1))
    opt = Adam(learning_rate=0.1)
    model.compile(optimizer=opt, loss='mse')

    # fit model
    model.fit(XTrain, yTrain, epochs=1000, verbose=0)

    yPred = list(model.predict(XValid, verbose=0))
    yPredList = []
    for i in range(len(yPred)):
        yPredList.append(yPred[i][0])

#     for yTrue, yPred in zip(yTest, yPredList):
#         print(yTrue, yPred)

    MAPE = meanAbsolutePercentageError(yValid, yPredList)
    print(seed, MAPE)
    if MAPE < bestValidMAPE:
        print('Updating best MAPE to {}...'.format(MAPE))
        bestValidMAPE = MAPE
        print('Updating best seed to {}...'.format(seed))
        bestSeed = seed

# define model
print('Training model with best seed...')
tf.random.set_seed(seed=bestSeed)
model = Sequential()
model.add(Bidirectional(LSTM(nNeurons, activation='relu'), input_shape=(nDaysMin, nFeatures)))
model.add(Dense(1))
opt = Adam(learning_rate=0.1)
model.compile(optimizer=opt, loss='mse')

# fit model
model.fit(XTrain, yTrain, epochs=1000, verbose=0)

yPred = list(model.predict(XTest, verbose=0))
yPredList = []
for i in range(len(yPred)):
    yPredList.append(yPred[i][0])
    
MAPE = meanAbsolutePercentageError(yTest, yPredList)
print('Test MAPE:', MAPE)
MdSA = medianSymmetricAccuracy(yTest, yPredList)
print('Test MdSA:', MdSA)

0 2.8552057960128217
Updating best MAPE to 2.8552057960128217...
Updating best seed to 0...
1 2.784453642656162
Updating best MAPE to 2.784453642656162...
Updating best seed to 1...
2 0.4361258784078639
Updating best MAPE to 0.4361258784078639...
Updating best seed to 2...
3 0.3437415864201025
Updating best MAPE to 0.3437415864201025...
Updating best seed to 3...
4 18.707490151014603
5 2.3083822485462475
6 99.93231162129533
7 0.27410565647105894
Updating best MAPE to 0.27410565647105894...
Updating best seed to 7...
8 0.21023844176707762
Updating best MAPE to 0.21023844176707762...
Updating best seed to 8...
9 1.7312431503025423
10 2.1096548996724183
11 0.2844088066877513
12 99.91125971313618
13 2.4797664441603646
14 1.5994044938622312
15 1.7043463394157716
16 6.019068954083277
17 2.814335436478055
18 nan
19 1.0888512495682674
20 2.907474847932914
21 0.8101670681126945
22 4.728702395038241
23 1.045195116239809
24 99.9704966933795
25 3.6102966935765717
26 nan
27 28.966856680757346
28 4.

# CNN LSTM

In [9]:
import tensorflow as tf
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Flatten
from tensorflow.compat.v1.keras.layers import TimeDistributed
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import MaxPooling1D

# Number of subsequences to break X into (we do 15 = 5x3, 5 subsequences of size 3 each)
nSeq = 5
nSteps = 3

# define model
nNeurons = 50
nFeatures = 1
nFilters = 64

bestValidMAPE = 100
bestSeed = -1

# Reshape input
XTrainCNN = XTrainList.reshape((XTrainList.shape[0], nSeq, nSteps, nFeatures))
XValidCNN = XValidList.reshape((XValidList.shape[0], nSeq, nSteps, nFeatures))
XTestCNN = XTestList.reshape((XTestList.shape[0], nSeq, nSteps, nFeatures))

# print(XTrainCNN.shape)
# print(XValidCNN.shape)
# print(XTestCNN.shape)

for seed in range(100):
    tf.random.set_seed(seed=seed)
    model = Sequential()
    model.add(TimeDistributed(Conv1D(filters=nFilters, kernel_size=1, activation='relu'), input_shape=(None, nSteps, nFeatures)))
    model.add(TimeDistributed(MaxPooling1D(pool_size=2)))
    model.add(TimeDistributed(Flatten()))
    model.add(LSTM(nNeurons, activation='relu'))
    model.add(Dense(1))
    opt = Adam(learning_rate=0.1)
    model.compile(optimizer=opt, loss='mse')

    # fit model
    model.fit(XTrainCNN, yTrain, epochs=1000, verbose=0)

    yPred = list(model.predict(XValidCNN, verbose=0))
    yPredList = []
    for i in range(len(yPred)):
        yPredList.append(yPred[i][0])

#     for yTrue, yPred in zip(yTest, yPredList):
#         print(yTrue, yPred)

    MAPE = meanAbsolutePercentageError(yValid, yPredList)
    print(seed, MAPE)
    if MAPE < bestValidMAPE:
        print('Updating best MAPE to {}...'.format(MAPE))
        bestValidMAPE = MAPE
        print('Updating best seed to {}...'.format(seed))
        bestSeed = seed

# define model
print('Training model with best seed...')
tf.random.set_seed(seed=bestSeed)
model = Sequential()
model.add(TimeDistributed(Conv1D(filters=nFilters, kernel_size=1, activation='relu'), input_shape=(None, nSteps, nFeatures)))
model.add(TimeDistributed(MaxPooling1D(pool_size=2)))
model.add(TimeDistributed(Flatten()))
model.add(LSTM(nNeurons, activation='relu'))
model.add(Dense(1))
opt = Adam(learning_rate=0.1)
model.compile(optimizer=opt, loss='mse')
# fit model
model.fit(XTrainCNN, yTrain, epochs=1000, verbose=0)

yPred = list(model.predict(XTestCNN, verbose=0))
yPredList = []
for i in range(len(yPred)):
    yPredList.append(yPred[i][0])
    
MAPE = meanAbsolutePercentageError(yTest, yPredList)
print('Test MAPE:', MAPE)
MdSA = medianSymmetricAccuracy(yTest, yPredList)
print('Test MdSA:', MdSA)

0 43.218178056886956
Updating best MAPE to 43.218178056886956...
Updating best seed to 0...
1 1.6321928397321808
Updating best MAPE to 1.6321928397321808...
Updating best seed to 1...
2 32.931458940895084
3 1.5731657614143166
Updating best MAPE to 1.5731657614143166...
Updating best seed to 3...
4 0.9672983828400431
Updating best MAPE to 0.9672983828400431...
Updating best seed to 4...
5 1.7595859172875175
6 2.7580810900561037
7 39.06412797879245
8 45.60552266107496
9 7.09523618792279
10 1.2376491833037715
11 36.86693024486889
12 1.0808414759031901
13 0.3918293141316104
Updating best MAPE to 0.3918293141316104...
Updating best seed to 13...
14 3.0686877552411174
15 6.40020133377054
16 40.65451206107428
17 41.907059562354476
18 4.221222172342194
19 42.01263718298199
20 6.657425129789056
21 0.8192383264525736
22 41.52533559989733
23 1.8467598622217924
24 43.6080202314321
25 5.88265859103676
26 3.757758930347135
27 6.69691976815707
28 39.583393886876785
29 1.6098575874838785
30 42.7815154

# ConvLSTM

In [10]:
from tensorflow.keras.layers import ConvLSTM2D

# Number of subsequences to break X into (we do 15 = 5x3, 5 subsequences of size 3 each)
nSeq = 5
nSteps = 3
# Each input is rows x columns, we have rows=1 and columns=nSteps

# define model
nNeurons = 50
nFeatures = 1
nFilters = 64

bestValidMAPE = 100
bestSeed = -1

# Reshape input
XTrainConv = XTrainList.reshape((XTrainList.shape[0], nSeq, 1, nSteps, nFeatures))
XValidConv = XValidList.reshape((XValidList.shape[0], nSeq, 1, nSteps, nFeatures))
XTestConv = XTestList.reshape((XTestList.shape[0], nSeq, 1, nSteps, nFeatures))

for seed in range(100):
    tf.random.set_seed(seed=seed)
    model = Sequential()
    model.add(ConvLSTM2D(filters=64, kernel_size=(1,2), activation='relu', input_shape=(nSeq, 1, nSteps, nFeatures)))
    model.add(Flatten())
    model.add(Dense(1))
    opt = Adam(learning_rate=0.1)
    model.compile(optimizer=opt, loss='mse')

    # fit model
    model.fit(XTrainConv, yTrain, epochs=1000, verbose=0)

    yPred = list(model.predict(XValidConv, verbose=0))
    yPredList = []
    for i in range(len(yPred)):
        yPredList.append(yPred[i][0])

#     for yTrue, yPred in zip(yTest, yPredList):
#         print(yTrue, yPred)

    MAPE = meanAbsolutePercentageError(yValid, yPredList)
    print(seed, MAPE)
    if MAPE < bestValidMAPE:
        print('Updating best MAPE to {}...'.format(MAPE))
        bestValidMAPE = MAPE
        print('Updating best seed to {}...'.format(seed))
        bestSeed = seed

# define model
print('Training model with best seed...')
tf.random.set_seed(seed=bestSeed)
model = Sequential()
model.add(ConvLSTM2D(filters=64, kernel_size=(1,2), activation='relu', input_shape=(nSeq, 1, nSteps, nFeatures)))
model.add(Flatten())
model.add(Dense(1))
opt = Adam(learning_rate=0.1)
model.compile(optimizer=opt, loss='mse')
# fit model
model.fit(XTrainConv, yTrain, epochs=1000, verbose=0)

yPred = list(model.predict(XTestConv, verbose=0))
yPredList = []
for i in range(len(yPred)):
    yPredList.append(yPred[i][0])
    
MAPE = meanAbsolutePercentageError(yTest, yPredList)
print('Test MAPE:', MAPE)
MdSA = medianSymmetricAccuracy(yTest, yPredList)
print('Test MdSA:', MdSA)

0 0.3490946199669345
Updating best MAPE to 0.3490946199669345...
Updating best seed to 0...
1 0.572631827824
2 2.6905338329904125
3 0.23536122221918337
Updating best MAPE to 0.23536122221918337...
Updating best seed to 3...
4 3.4835046738740036
5 99.88545238357511
6 0.36577077521018514
7 5.149376540352869
8 0.39022996337921617
9 3.496361754885692
10 1.153111019765104
11 2.0092768417062956
12 0.5550854930075385
13 0.9982912684915388
14 1.2635501664875144
15 99.88628076478159
16 0.2388274252007939
17 0.5625511064055874
18 1.499908850843924
19 99.914192936205
20 0.514389165332593
21 0.19810214718050315
Updating best MAPE to 0.19810214718050315...
Updating best seed to 21...
22 2.9973431747417534
23 99.88680830100473
24 0.5448846088575564
25 0.2553793744634029
26 99.88724314948237
27 99.88801620358936
28 0.44654311528511914
29 3.546119658087906
30 99.88443590441895
31 0.5512221936519195
32 0.4374062164016792
33 0.36846240455253715
34 0.7055360247478444
35 4.036459962577602
36 0.72700954471